In [6]:
from data_visualization import create_horizontal_bar_chart
from leaderboard_utils import get_mario_leaderboard
import matplotlib.pyplot as plt
import json

# Load the rank data
with open('rank_data_03_25_2025.json', 'r') as f:
    rank_data = json.load(f)

# Get the leaderboard data
df = get_mario_leaderboard(rank_data)

# Create the bar chart
game_name = "Super Mario Bros"
fig = create_horizontal_bar_chart(df, game_name)

# Save the figure in current directory
fig.savefig('mario_performance.png', bbox_inches='tight', dpi=300)

# Display the chart
plt.show()

C:\Users\yuxua\AppData\Local\Temp\ipykernel_14096\1296094769.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [17]:
from leaderboard_utils import get_combined_leaderboard, GAME_ORDER
from data_visualization import create_radar_charts  # to access the normalization function
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json

# Load the rank data
with open('rank_data_03_25_2025.json', 'r') as f:
    rank_data = json.load(f)

# Get combined leaderboard with all games
selected_games = {game: True for game in GAME_ORDER}
df = get_combined_leaderboard(rank_data, selected_games)

# Create figure and axis with better styling
sns.set_style("whitegrid")
plt.figure(figsize=(16, 9))
ax = plt.gca()

# Get unique models
models = df['Player'].unique()
n_games = len(GAME_ORDER)

# Define a professional color palette
colors = ['#2E86C1', '#E74C3C', '#27AE60', '#F39C12', '#8E44AD',
          '#16A085', '#D35400', '#2980B9', '#C0392B', '#2ECC71']

def normalize_values(values, mean, std):
    """
    Normalize values using z-score and scale to 0-100 range
    """
    if std == 0:
        return [50 if v > 0 else 0 for v in values]  # Handle zero std case
    z_scores = [(v - mean) / std for v in values]
    # Scale z-scores to 0-100 range, with mean at 50
    scaled_values = [max(0, min(100, (z * 30) + 50)) for z in z_scores]
    return scaled_values

# Calculate normalized scores for each game
for game_idx, game in enumerate(GAME_ORDER):
    # Get all scores for this game
    game_scores = []
    for model in models:
        score = df[df['Player'] == model][f'{game} Score'].values[0]
        if score != '_' and float(score) > 0:  # Only include non-zero scores
            game_scores.append((model, float(score)))
    
    # Sort scores from highest to lowest
    game_scores.sort(key=lambda x: x[1], reverse=True)
    
    # Extract sorted models and scores
    sorted_models = [x[0] for x in game_scores]
    scores = [x[1] for x in game_scores]
    
    # Calculate mean and std for normalization
    mean = np.mean(scores)
    std = np.std(scores)
    
    # Normalize scores using the function from data_visualization
    normalized_scores = normalize_values(scores, mean, std)
    
    # Calculate bar width based on number of models in this game
    n_models_in_game = len(sorted_models)
    bar_width = 0.8 / n_models_in_game
    
    # Plot bars for each model
    for i, (model, score) in enumerate(zip(sorted_models, normalized_scores)):
        ax.bar(game_idx + i*bar_width, score, 
              width=bar_width, 
              label=model if game_idx == 0 else "",  # Only add label for first game
              color=colors[i % len(colors)],
              alpha=0.8)

# Customize the plot
ax.set_xticks(np.arange(n_games))
ax.set_xticklabels(GAME_ORDER, rotation=45, ha='right', fontsize=10)
ax.set_ylabel('Normalized Performance Score', fontsize=12)
ax.set_title('AI Model Performance Comparison Across Gaming Tasks\n(Showing only participating models for each game)', 
             fontsize=14, pad=20)

# Add grid lines
ax.grid(True, axis='y', linestyle='--', alpha=0.3)

# Create legend with unique entries
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), 
          bbox_to_anchor=(1.05, 1), 
          loc='upper left',
          fontsize=9,
          title='AI Models',
          title_fontsize=10)

# Add a note about the visualization
plt.figtext(0.5, 0.01, 
           'Note: Scores are normalized within each game. Only models that participated in each game are shown.',
           ha='center', fontsize=9, style='italic')

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Save the figure
plt.savefig('ai_model_gaming_performance_clean.png', bbox_inches='tight', dpi=300)

# Show the plot
plt.show()

C:\Users\yuxua\AppData\Local\Temp\ipykernel_14096\1418582928.py:107: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
